In [1]:
%run init.ipynb

matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
preprocessor = mz.models.ArcI.get_default_preprocessor()

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 5533.59it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:05<00:00, 3640.01it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 1345756.13it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 6925.68it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:05<00:00, 3325.18it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 671055.87it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 7683.26it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<00

In [4]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f0d07559710>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f0d074e39b0>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [5]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [6]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)

In [7]:
padding_callback = mz.models.ArcI.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    sort=False,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    callback=padding_callback
)

In [8]:
model = mz.models.ArcI()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['left_length'] = 10
model.params['right_length'] = 100
model.params['left_filters'] = [128]
model.params['left_kernel_sizes'] = [3]
model.params['left_pool_sizes'] = [4]
model.params['right_filters'] = [128]
model.params['right_kernel_sizes'] = [3]
model.params['right_pool_sizes'] = [4]
model.params['conv_activation_func'] = 'relu'
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 100
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'relu'
model.params['dropout_rate'] = 0.9

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

ArcI(
  (embedding): Embedding(30058, 300)
  (conv_left): Sequential(
    (0): Sequential(
      (0): ConstantPad1d(padding=(0, 2), value=0)
      (1): Conv1d(300, 128, kernel_size=(3,), stride=(1,))
      (2): ReLU()
      (3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (conv_right): Sequential(
    (0): Sequential(
      (0): ConstantPad1d(padding=(0, 2), value=0)
      (1): Conv1d(300, 128, kernel_size=(3,), stride=(1,))
      (2): ReLU()
      (3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (dropout): Dropout(p=0.9, inplace=False)
  (mlp): Sequential(
    (0): Sequential(
      (0): Linear(in_features=3456, out_features=100, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=100, out_features=1, bias=True)
      (1): ReLU()
    )
  )
  (out): Linear(in_features=1, out_features=1, bias=True)
)
Trainable params:  9593859


In [9]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [10]:
trainer.run()

[Iter-102 Loss-0.964]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.45 - normalized_discounted_cumulative_gain@5(0.0): 0.5133 - mean_average_precision(0.0): 0.4813



[Iter-204 Loss-0.841]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4753 - normalized_discounted_cumulative_gain@5(0.0): 0.5391 - mean_average_precision(0.0): 0.5



[Iter-306 Loss-0.748]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5223 - normalized_discounted_cumulative_gain@5(0.0): 0.5795 - mean_average_precision(0.0): 0.539



[Iter-408 Loss-0.679]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5422 - normalized_discounted_cumulative_gain@5(0.0): 0.6112 - mean_average_precision(0.0): 0.5668



[Iter-510 Loss-0.629]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.587 - normalized_discounted_cumulative_gain@5(0.0): 0.6444 - mean_average_precision(0.0): 0.6056



[Iter-612 Loss-0.524]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6074 - normalized_discounted_cumulative_gain@5(0.0): 0.6633 - mean_average_precision(0.0): 0.6192



[Iter-714 Loss-0.455]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5888 - normalized_discounted_cumulative_gain@5(0.0): 0.6461 - mean_average_precision(0.0): 0.6037



[Iter-816 Loss-0.373]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5676 - normalized_discounted_cumulative_gain@5(0.0): 0.6271 - mean_average_precision(0.0): 0.5816



[Iter-918 Loss-0.284]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5689 - normalized_discounted_cumulative_gain@5(0.0): 0.6237 - mean_average_precision(0.0): 0.5832



[Iter-1020 Loss-0.227]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5428 - normalized_discounted_cumulative_gain@5(0.0): 0.6196 - mean_average_precision(0.0): 0.565

Cost time: 2189.8169510364532s
